# Delphes Samples

## Import modules

In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from HtautauRegression.dataset import H5Dataset
from HtautauRegression.helpers import plot_mass, find_indices, find_sample_number, make_four_vector

## Load samples

This is done using the `H5Dataset` class. 
Have both lephad decays and hadhad decays available 

In [ ]:
dname = "/bundle/data/ATLAS/gwilliam/ProjectSamples/h5/Delphes"
#lhsamples = sorted(glob(f"{dname}/LepHad/*lephad.h5"))
hhsamples = sorted(glob(f"{dname}/HadHad/*hadhad.h5"))
data = H5Dataset(hhsamples, target_name = "Mtautau") 

### Explore dataset

List of input feature names

In [ ]:
print(data.feature_names)

Input data, consisting of one row per event, each with the different features as columns

In [ ]:
X = data.X()
print(X.shape)
X

Plot an example e.g. $p_T$ of the first tau, using the index to get hte correct feature for all events (:)

In [ ]:
itau1pt = data.feature_names.index("Tau1_Pt")

plt.figure()
plt.title("Distribution of first input feature")
plt.xlabel(r"$p_T (\tau_1)$ [GeV]")
plt.ylabel("Arbitrary Units")
plt.hist(X[:, itau1pt], bins = np.linspace(0, 1500, 25), fill = None, 
         histtype = "step", density = True)
plt.yscale('log')
plt.show()

Output target mass for all samples

In [ ]:
y = data.y()
print (y.shape)
y

Plot this to see it covers a wide range of masses to avoid biasing 

In [ ]:
plt.figure()
plt.title("Distribution of output masses for all samples")
plt.xlabel(r"$m_{\tau\tau}$ [GeV]")
plt.ylabel("Arbitrary Units")
plt.hist(y, bins = np.linspace(50, 350, 60), fill = None, 
         histtype = "step", density = True)
plt.show()

Auxilary data, including a label for the individual samples and the ATLAS MMC and CMS SVFIt to compare to 

In [ ]:
print(data.aux_labels())
isvf = data.aux_labels().index("MSVFit")
immc = data.aux_labels().index("MMMC")
aux = data.aux()
aux

Create the four-vectors for the two taus using the helper functions to get just the 125 GeV data and create the four vectors. Add them to get the HH four-vector

In [ ]:
i125 = find_indices(hhsamples, aux, data.aux_labels(), "125")

tau1 = make_four_vector("Tau1", data, i125)
tau2 = make_four_vector("Tau2", data, i125)
hh = tau1 + tau2

Plot the visible mass vs the truth mass, where you can see that it peaks below 125 GeV due to the energy lost via the neutrinos and is broad.  

In [ ]:
plot_mass(None, mtrue=y[i125], mvis = hh.mass, 
          title = "Mass (125 GeV)", bins=np.linspace(100, 200, 50), true_scale = 0.1)

ATLAS and CMS have both created algorithms to try to correct for the energy loss due to neutrinos.  Let's plot these as well

In [ ]:
plot_mass(None, mtrue=y[i125], mvis=hh.mass, mmmc = aux[i125, immc], msv = aux[i125, isvf],
          title = "Mass (125 GeV)", bins=np.linspace(100, 200, 50), true_scale = 0.1)

You can see that they correct the position but still don't have a very good resolution.  The aim of this project is to train an MVA to do better at reconstructing the mass.

### Get individual data events
Data for first event

In [ ]:
X0, y0 = data[0]
print(X0)
print(y0)